In [1]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from keras.metrics import top_k_categorical_accuracy
def top_3_accuracy(x,y): return top_k_categorical_accuracy(x,y, 3)
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from glob import glob
import gc
from tqdm import tqdm
gc.enable()
def get_available_gpus():
    from tensorflow.python.client import device_lib
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

Using TensorFlow backend.


['/device:GPU:0']

In [2]:
CLASS_NAME=\
['The_Eiffel_Tower', 'The_Great_Wall_of_China', 'The_Mona_Lisa', 'airplane', 'alarm_clock', 'ambulance', 'angel',
 'animal_migration', 'ant', 'anvil', 'apple', 'arm', 'asparagus', 'axe', 'backpack', 'banana', 'bandage', 'barn',
 'baseball', 'baseball_bat', 'basket', 'basketball', 'bat', 'bathtub', 'beach', 'bear', 'beard', 'bed', 'bee',
 'belt', 'bench', 'bicycle', 'binoculars', 'bird', 'birthday_cake', 'blackberry', 'blueberry', 'book',
 'boomerang', 'bottlecap', 'bowtie', 'bracelet', 'brain', 'bread', 'bridge', 'broccoli', 'broom',
 'bucket', 'bulldozer', 'bus', 'bush', 'butterfly', 'cactus', 'cake', 'calculator', 'calendar', 'camel',
 'camera', 'camouflage', 'campfire', 'candle', 'cannon', 'canoe', 'car', 'carrot', 'castle', 'cat', 'ceiling_fan',
 'cell_phone', 'cello', 'chair', 'chandelier', 'church', 'circle', 'clarinet', 'clock', 'cloud', 'coffee_cup',
 'compass', 'computer', 'cookie', 'cooler', 'couch', 'cow', 'crab', 'crayon', 'crocodile', 'crown', 'cruise_ship',
 'cup', 'diamond', 'dishwasher', 'diving_board', 'dog', 'dolphin', 'donut', 'door', 'dragon', 'dresser',
 'drill', 'drums', 'duck', 'dumbbell', 'ear', 'elbow', 'elephant', 'envelope', 'eraser', 'eye', 'eyeglasses',
 'face', 'fan', 'feather', 'fence', 'finger', 'fire_hydrant', 'fireplace', 'firetruck', 'fish', 'flamingo',
 'flashlight', 'flip_flops', 'floor_lamp', 'flower', 'flying_saucer', 'foot', 'fork', 'frog', 'frying_pan',
 'garden', 'garden_hose', 'giraffe', 'goatee', 'golf_club', 'grapes', 'grass', 'guitar', 'hamburger',
 'hammer', 'hand', 'harp', 'hat', 'headphones', 'hedgehog', 'helicopter', 'helmet', 'hexagon', 'hockey_puck',
 'hockey_stick', 'horse', 'hospital', 'hot_air_balloon', 'hot_dog', 'hot_tub', 'hourglass', 'house', 'house_plant',
 'hurricane', 'ice_cream', 'jacket', 'jail', 'kangaroo', 'key', 'keyboard', 'knee', 'ladder', 'lantern', 'laptop',
 'leaf', 'leg', 'light_bulb', 'lighthouse', 'lightning', 'line', 'lion', 'lipstick', 'lobster', 'lollipop', 'mailbox',
 'map', 'marker', 'matches', 'megaphone', 'mermaid', 'microphone', 'microwave', 'monkey', 'moon', 'mosquito',
 'motorbike', 'mountain', 'mouse', 'moustache', 'mouth', 'mug', 'mushroom', 'nail', 'necklace', 'nose', 'ocean',
 'octagon', 'octopus', 'onion', 'oven', 'owl', 'paint_can', 'paintbrush', 'palm_tree', 'panda', 'pants',
 'paper_clip', 'parachute', 'parrot', 'passport', 'peanut', 'pear', 'peas', 'pencil', 'penguin', 'piano',
 'pickup_truck', 'picture_frame', 'pig', 'pillow', 'pineapple', 'pizza', 'pliers', 'police_car', 'pond',
 'pool', 'popsicle', 'postcard', 'potato', 'power_outlet', 'purse', 'rabbit', 'raccoon', 'radio', 'rain',
 'rainbow', 'rake', 'remote_control', 'rhinoceros', 'river', 'roller_coaster', 'rollerskates', 'sailboat',
 'sandwich', 'saw', 'saxophone', 'school_bus', 'scissors', 'scorpion', 'screwdriver', 'sea_turtle', 'see_saw',
 'shark', 'sheep', 'shoe', 'shorts', 'shovel', 'sink', 'skateboard', 'skull', 'skyscraper', 'sleeping_bag',
 'smiley_face', 'snail', 'snake', 'snorkel', 'snowflake', 'snowman', 'soccer_ball', 'sock', 'speedboat',
 'spider', 'spoon', 'spreadsheet', 'square', 'squiggle', 'squirrel', 'stairs', 'star', 'steak', 'stereo',
 'stethoscope', 'stitches', 'stop_sign', 'stove', 'strawberry', 'streetlight', 'string_bean', 'submarine',
 'suitcase', 'sun', 'swan', 'sweater', 'swing_set', 'sword', 't-shirt', 'table', 'teapot', 'teddy-bear',
 'telephone', 'television', 'tennis_racquet', 'tent', 'tiger', 'toaster', 'toe', 'toilet', 'tooth',
 'toothbrush', 'toothpaste', 'tornado', 'tractor', 'traffic_light', 'train', 'tree', 'triangle',
 'trombone', 'truck', 'trumpet', 'umbrella', 'underwear', 'van', 'vase', 'violin', 'washing_machine',
 'watermelon', 'waterslide', 'whale', 'wheel', 'windmill', 'wine_bottle', 'wine_glass', 'wristwatch',
 'yoga', 'zebra', 'zigzag']

word_encoder = LabelEncoder()
word_encoder.fit(CLASS_NAME)

LabelEncoder()

In [3]:
def softmax(X, theta=1.0, axis=None):
    """
    Compute the softmax of each element along an axis of X.

    Parameters
    ----------
    X: ND-Array. Probably should be floats.
    theta (optional): float parameter, used as a multiplier
        prior to exponentiation. Default = 1.0
    axis (optional): axis to compute values along. Default is the
        first non-singleton axis.

    Returns an array the same size as X. The result will sum to 1
    along the specified axis.
    """

    # make X at least 2d
    y = np.atleast_2d(X)

    # find axis
    if axis is None:
        axis = next(j[0] for j in enumerate(y.shape) if j[1] > 1)

    # multiply y against the theta parameter,
    y = y * float(theta)

    # subtract the max for numerical stability
    y = y - np.expand_dims(np.max(y, axis=axis), axis)

    # exponentiate y
    y = np.exp(y)

    # take the sum along the specified axis
    ax_sum = np.expand_dims(np.sum(y, axis=axis), axis)

    # finally: divide elementwise
    p = y / ax_sum

    # flatten if X was 1D
    if len(X.shape) == 1: p = p.flatten()

    return p

In [4]:
log_dir = "./predict_numpy/"
logits_resnet34 = []
for best in [3, 4, 5, 6, 7]:
    logit = np.load(f"{log_dir}/resnet34_embedding_30k/dataset.predictions.infer.logits.satge1.{best}.npy")
    logit = softmax(logit, axis=1)
    logits_resnet34.append(logit)
logits_resnet34 = np.asarray(logits_resnet34)
logits_resnet34 = np.mean(logits_resnet34, axis=0)

logits_resnext = np.load(f"{log_dir}/phuong/resnext.npy")

logits_seresnet = []
for best in [1,2,3,4]:
    logit = np.load(f"{log_dir}/se_resnet50_full_30k/dataset.predictions.infer.logits.satge1.{best}.npy")
    logit = softmax(logit, axis=1)
    logits_seresnet.append(logit)
logits_seresnet = np.asarray(logits_seresnet)
logits_seresnet = np.mean(logits_seresnet, axis=0)

logits_seresnet_aphuong = np.load(f"{log_dir}/phuong/se_resnet_50.npy")

logits_resnet34_full = []
for best in [1,2,3,4,5]:
    logit = np.load(f"{log_dir}/fusion_50k_imgsize64/dataset.predictions.infer.logits.satge1.{best}.npy")
    logit = softmax(logit, axis=1)
    logits_resnet34_full.append(logit)
logits_resnet34_full = np.asarray(logits_resnet34_full)
logits_resnet34_full = np.mean(logits_resnet34_full, axis=0)


pred = (logits_resnet34_full + logits_resnext + logits_seresnet + logits_seresnet_aphuong) / 4
# pred = (pred + logits_resnet34_full) / 2

In [5]:
pred.shape

(112199, 340)

In [6]:
A = np.array([[0, 1, 2], [0, 2, 0]])
X = np.array([[0, 1, 2], [1, 2, 0], [2, 1, 2], [3, 2, 0]])
X

array([[0, 1, 2],
       [1, 2, 0],
       [2, 1, 2],
       [3, 2, 0]])

In [7]:
newrow = np.zeros(3) #[1,2,3]
np.vstack([X, newrow])

array([[0., 1., 2.],
       [1., 2., 0.],
       [2., 1., 2.],
       [3., 2., 0.],
       [0., 0., 0.]])

In [8]:
pred_340 = np.vstack([pred, np.zeros(340)])
pred_340.shape

(112200, 340)

In [9]:
pred_340 *= 1000000000

In [10]:
pred_340 = pred_340.astype(np.int64)
pred_340


array([[        4,         8,         1, ...,       148,         5,
               20],
       [      224,     10188,       178, ...,      3100,        83,
              120],
       [     8656, 943073272,       148, ...,       236,       169,
             2471],
       ...,
       [       31,        60,       280, ...,        57,        41,
               94],
       [      617,      1064,       765, ...,       496,       504,
              792],
       [        0,         0,         0, ...,         0,         0,
                0]])

In [11]:
word_encoder.classes_[:3]

array(['The_Eiffel_Tower', 'The_Great_Wall_of_China', 'The_Mona_Lisa'],
      dtype='<U23')

In [12]:
from ortools.graph import pywrapgraph
def mcf_balance(pred_340):
    m = pred_340.astype(np.int64)
    size = pred_340.shape[0]
    print(size)
    mcf = pywrapgraph.SimpleMinCostFlow()
    for j in range(340):
        mcf.SetNodeSupply(j+size, size//340)
    for i in range(size):
        mcf.SetNodeSupply(i, -1)
        for j in range(340):
            mcf.AddArcWithCapacityAndUnitCost(j+size, i, 1, int(-m[i][j]))
    mcf.SolveMaxFlowWithMinCost()

    answ = np.zeros(size, dtype=np.int32)
    for i in range(mcf.NumArcs()):
        if mcf.Flow(i) > 0:
            answ[mcf.Head(i)] = mcf.Tail(i) - size
    return answ

rs = mcf_balance(pred_340)
rs

112200


array([237, 147,   1, ...,  42,  70, 214], dtype=int32)

In [13]:
opt_pred = word_encoder.classes_[rs]

In [14]:
# Top2: remove the first assignments (top1) and run the optimization module again

In [15]:
for idx, j in enumerate(rs):
    pred_340[idx, j] = 0
    
rs2 = mcf_balance(pred_340)
rs2

112200


array([284,  39,  65, ...,  50, 310, 173], dtype=int32)

In [16]:
opt_pred2 = word_encoder.classes_[rs2]

In [17]:
opt_pred.shape, opt_pred2.shape

((112200,), (112200,))

In [18]:
unique, counts = np.unique(opt_pred, return_counts=True)

print(np.asarray((unique, counts)).T[:3])

[['The_Eiffel_Tower' '330']
 ['The_Great_Wall_of_China' '330']
 ['The_Mona_Lisa' '330']]


In [19]:
top_3_pred = [word_encoder.classes_[np.argsort(-1*c_pred)[:3]] for c_pred in pred]

In [20]:
opt_pred2

array(['stereo', 'bottlecap', 'castle', ..., 'bush', 'toilet', 'line'],
      dtype='<U23')

In [21]:
final = []
for o, o2, t3 in zip(opt_pred, opt_pred2, top_3_pred):
    t2 = [c for c in t3 if (c != o) and (c != o2)]
    f3 = [o, o2] + t2[:1]
    final.append(f3)
    
c = 0
for o3, t3 in zip(final, top_3_pred):
    if not np.array_equal(o3,t3):
        print(t3, "=>", o3)
        c += 1
        if c > 5: 
            break

['mountain' 'tent' 'The_Eiffel_Tower'] => ['mountain', 'The_Eiffel_Tower', 'tent']
['submarine' 'swan' 'lobster'] => ['submarine', 'key', 'swan']
['bracelet' 'arm' 'hand'] => ['bracelet', 'hand', 'arm']
['octopus' 'octagon' 'snowflake'] => ['octopus', 'snowflake', 'octagon']
['stove' 'oven' 'cooler'] => ['stove', 'table', 'oven']
['steak' 'stereo' 'skull'] => ['steak', 'skull', 'stereo']


In [22]:
len(final)

112199

In [23]:
final_3_pred = [' '.join([col.replace(' ', '_') for col in row]) for row in final]
final_3_pred[:3]

['radio stereo binoculars',
 'hockey_puck bottlecap pool',
 'The_Great_Wall_of_China castle fence']

In [25]:
test_path = os.path.join('./predict_numpy/test_simplified.csv')
test_path
sub_df = pd.read_csv(test_path)

In [26]:
sub_df['word'] = final_3_pred
sub_df[['key_id', 'word']].to_csv('submission_opt_top2.csv', index=False)
sub_df.head(3)

,key_id,countrycode,drawing,word
0,9000003627287624,DE,"[[[17, 18, 20, 25, 137, 174, 242, 249, 251, 25...",radio stereo binoculars
1,9000010688666847,UA,"[[[174, 145, 106, 38, 11, 4, 4, 15, 29, 78, 16...",hockey_puck bottlecap pool
2,9000023642890129,BG,"[[[0, 12, 14, 17, 16, 24, 55, 57, 60, 79, 82, ...",The_Great_Wall_of_China castle fence


In [27]:
pred.shape

(112199, 340)

In [28]:
word_encoder.classes_[:3]

array(['The_Eiffel_Tower', 'The_Great_Wall_of_China', 'The_Mona_Lisa'],
      dtype='<U23')

In [29]:

import pickle

pickle.dump((pred, word_encoder), open("to_be_sent.pkl", 'wb'))


tmp_pred, tmp_word_encoder = pickle.load(open("to_be_sent.pkl", 'rb'))
print(tmp_pred.shape)
tmp_word_encoder

(112199, 340)


LabelEncoder()